<a href="https://colab.research.google.com/github/PARTHIBAN-007/Hands-ON-LLMs/blob/main/7_Advanced_Text_Generation_Techniques_and_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background-color:#ffecd2; color:#ad5389; text-align:center; padding:15px; border-radius:25px;">Advanced Text Generation </div>

In [ ]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp310-cp310-linux_x86_64.whl size=55265662 sha256=5492287a5ef10e5b7dad576b14b78285c531e93cdceeec8756692e545987d2ca
  Stored in directory: /root/.cache/pip/wheels/e4/18/46/58b5c613b17c8d000d79ae650980fe871b3b490e04e6faa1c1
Successfully built llama-cpp-python


# Loading an LLM

In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-12-20 10:37:09--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.118, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1734950229&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDk1MDIyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQx

In [ ]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [ ]:
llm.invoke("Hi! My name is parthiban. What is 1 + 1?")

'\n<|assistant|> The answer to 1 + 1 is 2.\n\nIf you have any other questions or need further assistance, feel free to ask!'

### Chains

In [ ]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [ ]:
basic_chain = prompt | llm

In [ ]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Parthiban. What is  2+3?",
    }
)

" The sum of 2 and 3 is 5.\n\n**Explanation:**\nThe operation you've asked for is a basic arithmetic addition. When you add the number 2 to the number 3, you combine their quantities to get a total amount. In this case, combining two units with three more units gives us five units in total. Mathematically, it can be represented as:\n\n\n$$\n\n2 + 3 = 5\n\n$$"

### Multiple Chains

In [ ]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""

title_prompt = PromptTemplate(
    template=template,
    input_variables=["summary"]
  )


title = LLMChain(
    llm=llm,
    prompt=title_prompt,
    output_key="title")

In [ ]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Lingering Echoes: A Tale of Loss and Rediscovery"'}

In [ ]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template,
    input_variables=["summary", "title"]
)
character = LLMChain(
    llm=llm,
    prompt=character_prompt,
    output_key="character")

In [ ]:
llm_chain = title |character


llm_chain.invoke("a girl that lost her mother")


{'summary': 'a girl that lost her mother',
 'title': ' "Lingering Echoes: A Journey Through Grief"',
 'character': ' The protagonist, Lily, is a resilient and introspective young girl grappling with profound grief after losing her beloved mother at an early age. As she embarks on a transformative journey through emotional turmoil and self-discovery, Lily learns to navigate the lingering echoes of her loss while finding solace in cherished memories and newfound resilience.'}

In [ ]:
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template,
    input_variables=["summary", "title", "character"]
)
story = LLMChain(
    llm=llm,
    prompt=story_prompt,
    output_key="story")

In [ ]:
llm_chain = title | character | story

In [ ]:
llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Loss: A Mother\'s Absence in Little Sophia\'s Heart"',
 'character': " Little Sophia, a bright and resilient ten-year-old girl, carries an unwavering strength within her as she grapples with the profound void left by her mother's sudden passing; throughout the story, Sophia learns to navigate life's challenges while holding onto cherished memories of her loving and nurturing mother.\n\nSophia's heart is filled with a bittersweet blend of sorrow and gratitude as she reminisces about the tender moments spent with her dearly missed mother, whose warmth and love continue to guide Sophia through the complex journey of growing up without her constant presence in her life.",
 'story': " Whispers of Loss: A Mother's Absence in Little Sophia's Heart. In a quaint suburban neighborhood, ten-year-old Sophia grappled with an unwavering strength, navigating life’s turbulent tides after her mother's sudden departure. Amidst the pain 

# Memory

In [ ]:
basic_chain.invoke({"input_prompt": "Hi! My name is Parthiban. What is the value of pi?"})

" Hello, Parthiban! The value of π (pi) to a common approximation is 3.14159. However, it's an irrational number, which means its exact value cannot be expressed as a simple fraction and its decimal representation never ends or repeats. For most practical purposes, using 3.14 or 22/7 can suffice, but for more precise calculations, you may use the digits of pi to as many decimal places as needed."

In [ ]:
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your personal details, including your name, as I don't have the ability to access personal data. If you need assistance with something specific, feel free to ask!"

## ConversationBuffer

In [ ]:
template = """
<s><|user|>Current conversation:{chat_history}
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [ ]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-22-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [ ]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Parthiban. What is the value of pi?"})

{'input_prompt': 'Hi! My name is Parthiban. What is the value of pi?',
 'chat_history': '',
 'text': " Hello Parthiban, pi (π) is an irrational number representing the ratio of a circle's circumference to its diameter, approximately equal to 3.14159. However, it has infinitely many decimal places and cannot be precisely represented by a simple fraction or decimal.\n\nHere are some common approximations:\n- π ≈ 3.14\n- π ≈ 22/7 (an approximation)\n\nRemember that the more decimal places you need, you can use pi's digits to calculate it accurately as needed!"}

In [ ]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Parthiban. What is the value of pi?\nAI:  Hello Parthiban, pi (π) is an irrational number representing the ratio of a circle's circumference to its diameter, approximately equal to 3.14159. However, it has infinitely many decimal places and cannot be precisely represented by a simple fraction or decimal.\n\nHere are some common approximations:\n- π ≈ 3.14\n- π ≈ 22/7 (an approximation)\n\nRemember that the more decimal places you need, you can use pi's digits to calculate it accurately as needed!",
 'text': ' Your name is Parthiban.\n\nAs for your question about pi, I provided an explanation of its value and some common approximations. Pi (π) is approximately 3.14159 but has infinitely many decimal places and cannot be precisely expressed as a fraction or a finite decimal.'}

## ConversationBufferMemoryWindow

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-25-9fb59949bdfd>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [ ]:
llm_chain.invoke({"input_prompt":"Hi! My name is Parthiban and i am 20 years old. What is the value of pi?"})
llm_chain.invoke({"input_prompt":"What is the value of square of pi?"})

{'input_prompt': 'What is the value of square of pi?',
 'chat_history': "Human: Hi! My name is Parthiban and i am 20 years old. What is the value of pi?\nAI:  Hello Parthiban! Pi, which is approximately equal to 3.14159, is a mathematical constant representing the ratio of a circle's circumference to its diameter. It is an irrational number, meaning it cannot be exactly expressed as a simple fraction and its decimal representation never ends or repeats.\n\nHere's how you can use pi in some basic calculations:\n\n- To calculate the circumference (C) of a circle with radius r or diameter d, you can use C = 2 * pi * r or C = pi * d.\n- The area (A) of a circle is given by A = pi * r^2, where r is the radius.\n\nRemember that there are many more applications and properties related to pi in mathematics and science!",
 'text': " The exact value of pi squared (pi^2) cannot be expressed as a simple fraction or decimal because pi itself is an irrational number. However, you can calculate an app

In [ ]:
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Parthiban and i am 20 years old. What is the value of pi?\nAI:  Hello Parthiban! Pi, which is approximately equal to 3.14159, is a mathematical constant representing the ratio of a circle's circumference to its diameter. It is an irrational number, meaning it cannot be exactly expressed as a simple fraction and its decimal representation never ends or repeats.\n\nHere's how you can use pi in some basic calculations:\n\n- To calculate the circumference (C) of a circle with radius r or diameter d, you can use C = 2 * pi * r or C = pi * d.\n- The area (A) of a circle is given by A = pi * r^2, where r is the radius.\n\nRemember that there are many more applications and properties related to pi in mathematics and science!\nHuman: What is the value of square of pi?\nAI:  The exact value of pi squared (pi^2) cannot be expressed as a simple fraction or decimal because pi itself is an irrational number. However, you ca

In [ ]:
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is the value of square of pi?\nAI:  The exact value of pi squared (pi^2) cannot be expressed as a simple fraction or decimal because pi itself is an irrational number. However, you can calculate an approximate numerical value for pi^2 using the approximation of pi that we mentioned earlier: 3.14159.\n\nHere's how to find the square of pi (pi^2) approximately:\n\npi^2 = (3.14159)^2 ≈ 9.8695877281\n\nKeep in mind that this is an approximate value, and more digits can be calculated using higher precision representations of pi if needed for specific applications.\nHuman: What is my name?\nAI:  Your name is Parthiban.\n\n-----EXAMPLES-----\nCurrent conversation:Human: What is the square root of 256?\nAI: The square root of 256 is approximately 16, as 16 * 16 = 256.\n\nCurrent conversation: Human: How do I calculate the area of a circle with radius 7 cm?\nAI: To calculate the area (A) of a circle with radius (r) of 7 centimete

## ConversationSummary

In [ ]:
# Create a summary prompt template
summary_prompt_template = """
<s><|user|>Summarize the conversations and update with the new lines.
Current summary:
{summary}
new lines of conversation:
{new_lines}
New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [ ]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-30-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is parthiban. what is the value of pi?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': " Parthiban inquired about the value of Pi (π), to which the AI responded that it is approximately 3.14159, an irrational number without a repeating decimal pattern. The AI also mentioned commonly used approximations like 22/7 and 3.14.\n\nNew summary with updated conversation:\nParthiban, who introduced himself as such, sought information on the value of Pi (π). The AI elucidated that Pi is approximately 3.14159 but clarified it's an irrational number meaning its decimal representation neither terminates nor repeats. It also highlighted common approximations used for practical purposes: 22/7 and 3.14.",
 'text': ' Your name, as mentioned in the updated summary of the conversation, is Parthiban.'}

In [ ]:
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Summary: Human, identified as Maarten in the context of this conversation, first asked about the sum of 1 + 1 and received an answer of 2 from the AI. Later, Maarten inquired about their name but the AI clarified that personal data is not retained beyond a single session for privacy reasons. The AI offered further assistance if needed.',
 'text': ' The first question you asked was "what\'s 1 + 1?"'}

In [ ]:
memory.load_memory_variables({})

{'chat_history': " Parthiban, introduced himself to the AI, asked about Pi (π) and learned that it's approximately 3.14159, an irrational number with a non-repeating decimal pattern. The AI also informed him of typical approximations like 22/7 and 3.14 for practical use. When inquired about his name by the AI, Parthiban confirmed that it was mentioned as such in the conversation summary."}